In [1]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
from ExtendedKalmanFilter import EKF

In [2]:
!pip install --upgrade tdmclient

import tdmclient.notebook
await tdmclient.notebook.start()

In [30]:
motor_left_target = 200
motor_right_target = 0


In [31]:
motor_left_target = 0
motor_right_target = 0

In [28]:
motor_right_speed

-117

In [ ]:
import asyncio

async def move_robot_and_get_speed(client):
    global node

    # Set motor targets
    v = {
        "motor.left.target": [int(100)],
        "motor.right.target": [int(100)],
    }
    await node.set_variables(v)

    # Wait for the movement to complete
    await asyncio.sleep(2)  # Adjust the sleep duration as needed

    # Read current speed values
    left_speed = await node.get_variable("motor.left.speed")
    right_speed = await node.get_variable("motor.right.speed")

    print("Left Speed:", left_speed)
    print("Right Speed:", right_speed)

    # Stop the robot
    v_stop = {
        "motor.left.target": [int(0)],
        "motor.right.target": [int(0)],
    }
    await node.set_variables(v_stop)

# Call the function within your A_star_move function or main logic
await move_robot_and_get_speed(client)

In [12]:
x_f = 1
y_f = 1
x_d = 0
y_d = 0

In [13]:
# provisoire pour filtre
measurements = np.array([
    [1,1,0],
    [1,0,90]
])

us = np.array([
    [1, 0],
    [0, 90]
])

-------------------------------------------------------------PID--------------------------------------------------------------

In [18]:
# Initialisation variable
conversion = 1
conversion_pixel_mm = 10
integral = 0.0
previous_error = 0.0
decalage = 0
timer_period = [100]

#Initialisation valeur non importante dans le programme final
l_speed = 50
r_speed = 50

motor_left_target  = 0
motor_right_target = 0

# initialisation du filtre
ekf = EKF(1, initial_pos, initial_speed)

def calculate_directional_angle_2D(point1, point2, point3):
    vector1 = np.array(point1) - np.array(point2)
    vector3 = np.array(point3) - np.array(point2)

    # Calculate angles in radians
    angle1 = np.arctan2(vector1[1], vector1[0])
    angle3 = np.arctan2(vector3[1], vector3[0])

    # Calculate directional angle
    directional_angle = angle3 - angle1

    # Adjust the angle to be between -pi and pi
    directional_angle = (directional_angle + np.pi) % (2 * np.pi) - np.pi

    return directional_angle

# Changer les valeurs global,vitesse motteur,
def Reglage():
    global l_speed, r_speed, integral, previous_error,conversion,x_f,y_f,x_d,y_d,x_t,y_t

    # Déclaration des constantes PID
    kp = 1.0  # Terme proportionnel
    ki = 0.1  # Terme intégral
    kd = 0.01  # Terme dérivé
    tolerance = 0.01  # tolerance
    
    # Calculate the hypotenuse of the vector (point2 - point3)
    Hypothenus = np.linalg.norm(np.array([x_d,y_d]) - np.array([x_t,y_t]))

    # Calculate the directional angle between point1, point2, and point3
    Angle = calculate_directional_angle_2D([x_f,y_f],[x_d,y_d],[x_t,y_t])

    # Calculate decalage using sin(Angle) * Hypotenuse
    decalage = np.sin(Angle) * Hypothenus  

    if abs(decalage) > tolerance:
        # Calcul des termes PID
        P = kp * decalage
        I = ki * integral
        D = kd * (decalage - previous_error)

        # Calcul de la commande finale
        output = P + I + D
        
        l_speed = 600+(output * conversion)  # Assuming conversion is defined
        r_speed = 600-(output*conversion)
       
        # Mettre à jour les variables pour la prochaine itération
        previous_error = decalage
        integral += decalage


#Boucle principal 

@onevent
def timer0():
    global motor_left_target, motor_right_target, l_speed, r_speed
    # call the filter to find position  
    x_t,y_t,_=ekf.filter(measurements[0], us[0])
    Reglage()
    # Set motor powers
    motor_left_target = round(l_speed) 
    motor_right_target = round(r_speed) 
        


Error in callback <function _post_run_cell at 0x7fa5f2a35ee0> (for post_run_cell):


TDMIncompatibleVarSizeError: Incompatible size (assignment of a list of size 1 to timer.period[2])

In [16]:
x,y,teta=ekf.filter(measurements[0], us[0])
y

State Estimate Before EKF=[ 2.355 -0.079 -0.028]
Observation=[1 0 0]
State Estimate After EKF=[ 1.477 -0.074 -0.035]


-0.0735994088756344

In [5]:
ekf.filter(measurements[0], us[0])

State Estimate Before EKF=[-0.20992708  0.32719976 34.53294203]
Observation=[1 0 0]
State Estimate After EKF=[ 0.4942384   0.08183813 13.17624931]
